# xml 파일 쪼개기

### 각 application이 하나의 xml 파일 그 자체    
__=> 하나씩 잘라야 함__

In [1]:
import os
import re
import xml.etree.ElementTree as et

os.chdir('J:/data')

In [ ]:
f=open('sample.xml', 'r')
endtag_regex = re.compile('^<!DOCTYPE (.*) SYSTEM')
endtag = ''
doc=''

In [ ]:
line = f.readline()
doc += line

In [ ]:
endtag

In [ ]:
endtag_regex.findall(line)

In [ ]:
endtag = endtag_regex.findall(line) if not endtag else endtag
endtag

In [ ]:
not endtag

In [ ]:
doc

In [ ]:
terminate = re.compile('^</{0}>'.format(endtag[0]))

In [ ]:
terminate.findall(line)

if not ~ else? 이게 뭐지?

In [ ]:
if not endtag == True:
    endtag = endtag_regex.findall(line)
else:
    endtag

endtag

아하 이렇게 쓰기 싫어서 그냥 한 줄로 끝냈구만!  
함수 이해 완료

In [6]:
def extract_xml_strings(filename):
    """
    Given a string [filename], opens the file and returns a generator
    that yields tuples. A tuple is of format (year, xmldoc string). A tuple
    is returned for every valid XML doc in [filename]
    """
    # search for terminating XML tag
    endtag_regex = re.compile('^<!DOCTYPE (.*) SYSTEM')
    endtag = ''
    with open(filename, 'r') as f:
        doc = ''  # initialize current XML doc to empty string
        for line in f:
            doc += line
            endtag = endtag_regex.findall(line) if not endtag else endtag   #is there any endtags?
            if not endtag:
                continue      #if there's no end tag, keep the loop going
            terminate = re.compile('^</{0}>'.format(endtag[0]))    #if there's a end tag, then check whether the doc is finished or not
            if terminate.findall(line):     #if there's a terminator
                yield (doc)                 #export doc and reinitialize a doc
                endtag = ''
                doc = ''

In [ ]:
sample = extract_xml_strings('sample.xml')

In [ ]:
for x in sample:
    print x

good~ 이젠 sample.next 해서 각 doc을 parsing, App Np, DN, assignee, class 추출하자

# 각 XML에서 원하는 element 뽑기

App NO, DN, Assignee, Class 필요

일단 각각 element가 뭔지 보면 (등장 순서대로)  
DN : us-bibliographic-data-application / document-id / doc-number  
APP No : us-bibliographic-data-application / application-reference / doc-number  
Class : us-bibliographic-data-application / classification-national / main-classification  
__** issue : 3digit에서 0 제거 안 함__  ex) 312/29.1 = 312029001  
Assignee : us-bibliographic-data-application / assignees / addressbook / orgname

In [ ]:
doc1 = sample.next()
doc2 = sample.next()

In [ ]:
root = et.fromstring(doc1)
root

In [ ]:
root.find('us-bibliographic-data-application')

list 만들어서 각 정보들 넣은 후, csv로 바꾸자  
dn, app no, assignee 담은 csv  
dn, app no, class 담은 csv  

In [ ]:
aline = []
cline = []

In [ ]:
# DN, APP NO 얻기
for dn in root.iter("doc-number"):
    aline.append(dn.text)
    cline.append(dn.text)
    
print aline, cline

In [ ]:
# Assignee 얻기
for ass in root.iter('orgname'):
    aline.append(ass.text)

aline.append('') if len(aline)==2 else aline
aline

In [ ]:
# Class 얻기
for ass in root.iter('main-classification'):
    rawclass = ass.text
    cline.append(str(int(rawclass[0:3])))
    if not int(rawclass[6:9]):
        cline.append(str(int(rawclass[3:6])))
    else:
        sub1 = str(int(rawclass[3:6]))
        sub2 = str(int(rawclass[6:9]))
        cline.append(sub1 + '.' + sub2)

cline

# xml 자동으로 불러오기

application xml 파일이 1주일 단위로 구성  
ex) ipab20070308, ipab20070315, ...  
규칙에 맞게 자동으로 불러오도록 만들자

먼저 날짜 다루기

In [2]:
import datetime as dt

date = '20070308'
date = dt.datetime.strptime(date,'%Y%m%d')
date

datetime.datetime(2007, 3, 8, 0, 0)

In [3]:
date += dt.timedelta(weeks=1)
date

datetime.datetime(2007, 3, 15, 0, 0)

In [4]:
date=date.strftime('%Y%m%d')
date

'20070315'

filename 만들기

In [5]:
filename = 'ipab'+ date + '.xml'
filename

'ipab20070315.xml'

In [7]:
sample = extract_xml_strings(filename)